# Working code

In [2]:
import pandas as pd
from datetime import timedelta
pd.set_option('display.max_columns', None)

# Load the data
sentinel_data_path = 'Sentinel2_Data.csv'
ground_data_path = 'Ground_Data.csv'

# Read the CSV files
sentinel_df = pd.read_csv(sentinel_data_path, parse_dates=['timestamp'])
ground_df = pd.read_csv(ground_data_path, parse_dates=['Timestamp (UTC+12:00)'], low_memory=False)

# Rename timestamp columns
sentinel_df.rename(columns={'timestamp': 'timestamp_sentinel2'}, inplace=True)
ground_df.rename(columns={'Timestamp (UTC+12:00)': 'timestamp_ground'}, inplace=True)

# Convert columns to appropriate data types
numeric_cols = [
    'Discharge_(m^3/s)', 'Lake_Height_(m)', 'PercentFull_Active_Lake_Storage_(%)',
    'Snow_Volume_Opuha_Catchment_(mm)', 'Turbidity_Buoy_(NTU)', 'Turbidity_Platform_(NTU)',
    'Water_Temp_Buoy_(degC)', 'Water_Temp_Platform_(degC)', 'WDir(Deg)', 'WSpd(m/s)',
    'GustDir(Deg)', 'GustSpd(m/s)', 'WindRun(Km)', 'Rain(mm)', 'Tdry(C)', 'TWet(C)',
    'RH(%)', 'Tmax(C)', 'Tmin(C)', 'Pmsl(hPa)', 'Pstn(hPa)'
]

for col in numeric_cols:
    ground_df[col] = pd.to_numeric(ground_df[col], errors='coerce')

# Separate ground data based on sampling intervals
three_hour_avg_cols = ['Discharge_(m^3/s)']
fifteen_min_avg_cols = [
    'Water_Temp_Buoy_(degC)', 'Water_Temp_Platform_(degC)',
    'Turbidity_Buoy_(NTU)', 'Turbidity_Platform_(NTU)'
]
daily_avg_cols = [
    'Lake_Height_(m)', 'PercentFull_Active_Lake_Storage_(%)', 'Snow_Volume_Opuha_Catchment_(mm)',
    'WDir(Deg)', 'WSpd(m/s)', 'GustDir(Deg)', 'GustSpd(m/s)', 'WindRun(Km)', 'Rain(mm)',
    'Tdry(C)', 'TWet(C)', 'RH(%)', 'Tmax(C)', 'Tmin(C)', 'Pmsl(hPa)', 'Pstn(hPa)'
]

# Function to find the closest ground measurement to a given Sentinel-2 timestamp and compute time difference
def find_closest_measurement_time(sentinel_time, ground_df, cols, time_window):
    start_time = sentinel_time - time_window
    end_time = sentinel_time + time_window
    filtered_ground_df = ground_df[(ground_df['timestamp_ground'] >= start_time) & (ground_df['timestamp_ground'] <= end_time)]
    if not filtered_ground_df.empty:
        closest_time = filtered_ground_df['timestamp_ground'].iloc[(filtered_ground_df['timestamp_ground'] - sentinel_time).abs().argsort()[:1]].iloc[0]
        time_diff = abs((closest_time - sentinel_time).total_seconds())  # Time difference in seconds
        closest_rows = filtered_ground_df[filtered_ground_df['timestamp_ground'] == closest_time][cols].iloc[0].to_dict()
        return closest_rows, time_diff
    else:
        return {col: None for col in cols}, None

# Define the time window for finer intervals (e.g., 1 hour)
time_window = timedelta(hours=3)

# Prepare a list to store merged data
merged_data = []

# Process each Sentinel-2 timestamp
for index, row in sentinel_df.iterrows():
    sentinel_time = row['timestamp_sentinel2']
    merged_row = row.to_dict()
    
    # Find closest measurements and time difference for three hour average and fifteen minute interval columns
    closest_three_hour, time_diff_three_hour = find_closest_measurement_time(sentinel_time, ground_df, three_hour_avg_cols, time_window)
    closest_fifteen_min, time_diff_fifteen_min = find_closest_measurement_time(sentinel_time, ground_df, fifteen_min_avg_cols, time_window)
    
    # Update the merged row with these measurements
    merged_row.update(closest_three_hour)
    merged_row.update(closest_fifteen_min)
    
    # Add the time difference to the merged row (use the time difference from the finer interval measurements)
    merged_row['Ground_Measurements_time_diff_(seconds)'] = time_diff_fifteen_min if time_diff_fifteen_min is not None else time_diff_three_hour
    
    merged_data.append(merged_row)

# Convert the merged data to a DataFrame
merged_df = pd.DataFrame(merged_data)

# Create date columns for merging daily averages
ground_df['date'] = ground_df['timestamp_ground'].dt.date
sentinel_df['date'] = sentinel_df['timestamp_sentinel2'].dt.date
merged_df['date'] = pd.to_datetime(merged_df['timestamp_sentinel2']).dt.date

# Calculate daily averages for relevant ground data
daily_avg_df = ground_df.groupby('date')[daily_avg_cols].mean().reset_index()

# Merge the daily averages with the Sentinel-2 data
merged_df = pd.merge(merged_df, daily_avg_df, on='date', how='left')

# Drop the 'date' column as it was only for merging purposes
merged_df.drop(columns=['date'], inplace=True)

# Save the merged dataframe to a CSV file
merged_df.to_csv('New_Merged_Data.csv', index=False)

print("Merging completed. The merged dataset is saved as 'New_Merged_Data.csv'")


Merging completed. The merged dataset is saved as 'New_Merged_Data.csv'


In [3]:

# Load the merged data
merged_data_path = 'New_Merged_Data.csv'
merged_df = pd.read_csv(merged_data_path)

# List of band prefixes to aggregate
bands = ['B2', 'B3', 'B4', 'B8', 'B8A', 'B11', 'B12']

# Compute averages for each band
for band in bands:
    band_cols = [col for col in merged_df.columns if f'_{band}' in col]
    merged_df[f'{band}_AVG'] = merged_df[band_cols].mean(axis=1)

# Compute averages for cs and cs_CDF
cs_cols = [col for col in merged_df.columns if '_cs' in col and '_cs_cdf' not in col]
cs_cdf_cols = [col for col in merged_df.columns if '_cs_cdf' in col]

merged_df['cs_AVG'] = merged_df[cs_cols].mean(axis=1)
merged_df['cs_cdf_AVG'] = merged_df[cs_cdf_cols].mean(axis=1)

# Save the updated dataframe to a new CSV file
updated_data_path = 'Updated_Merged_Data.csv'
merged_df.to_csv(updated_data_path, index=False)

print("Aggregation completed. The updated dataset is saved as 'Updated_Merged_Data.csv'")


Aggregation completed. The updated dataset is saved as 'Updated_Merged_Data.csv'


In [4]:
merged_df

,timestamp_sentinel2,point1_B2,point10_B2,point11_B2,point12_B2,point13_B2,point14_B2,point15_B2,point16_B2,point17_B2,point18_B2,point19_B2,point2_B2,point20_B2,point21_B2,point22_B2,point3_B2,point4_B2,point5_B2,point6_B2,point7_B2,point8_B2,point9_B2,point1_B3,point10_B3,point11_B3,point12_B3,point13_B3,point14_B3,point15_B3,point16_B3,point17_B3,point18_B3,point19_B3,point2_B3,point20_B3,point21_B3,point22_B3,point3_B3,point4_B3,point5_B3,point6_B3,point7_B3,point8_B3,point9_B3,point1_B4,point10_B4,point11_B4,point12_B4,point13_B4,point14_B4,point15_B4,point16_B4,point17_B4,point18_B4,point19_B4,point2_B4,point20_B4,point21_B4,point22_B4,point3_B4,point4_B4,point5_B4,point6_B4,point7_B4,point8_B4,point9_B4,point1_B8,point10_B8,point11_B8,point12_B8,point13_B8,point14_B8,point15_B8,point16_B8,point17_B8,point18_B8,point19_B8,point2_B8,point20_B8,point21_B8,point22_B8,point3_B8,point4_B8,point5_B8,point6_B8,point7_B8,point8_B8,point9_B8,point1_B8A,point10_B8A,point11_B8A,point12_B8A,point13_B8A,point14_B8A,point15_B8A,point16_B8A,point17_B8A,point18_B8A,point19_B8A,point2_B8A,point20_B8A,point21_B8A,point22_B8A,point3_B8A,point4_B8A,point5_B8A,point6_B8A,point7_B8A,point8_B8A,point9_B8A,point1_B11,point10_B11,point11_B11,point12_B11,point13_B11,point14_B11,point15_B11,point16_B11,point17_B11,point18_B11,point19_B11,point2_B11,point20_B11,point21_B11,point22_B11,point3_B11,point4_B11,point5_B11,point6_B11,point7_B11,point8_B11,point9_B11,point1_B12,point10_B12,point11_B12,point12_B12,point13_B12,point14_B12,point15_B12,point16_B12,point17_B12,point18_B12,point19_B12,point2_B12,point20_B12,point21_B12,point22_B12,point3_B12,point4_B12,point5_B12,point6_B12,point7_B12,point8_B12,point9_B12,point1_cs,point10_cs,point11_cs,point12_cs,point13_cs,point14_cs,point15_cs,point16_cs,point17_cs,point18_cs,point19_cs,point2_cs,point20_cs,point21_cs,point22_cs,point3_cs,point4_cs,point5_cs,point6_cs,point7_cs,point8_cs,point9_cs,point1_cs_cdf,point10_cs_cdf,point11_cs_cdf,point12_cs_cdf,point13_cs_cdf,point14_cs_cdf,point15_cs_cdf,point16_cs_cdf,point17_cs_cdf,point18_cs_cdf,point19_cs_cdf,point2_cs_cdf,point20_cs_cdf,point21_cs_cdf,point22_cs_cdf,point3_cs_cdf,point4_cs_cdf,point5_cs_cdf,point6_cs_cdf,point7_cs_cdf,point8_cs_cdf,point9_cs_cdf,sun_zenith,sun_azimuth,Discharge_(m^3/s),Water_Temp_Buoy_(degC),Water_Temp_Platform_(degC),Turbidity_Buoy_(NTU),Turbidity_Platform_(NTU),Ground_Measurements_time_diff_(seconds),Lake_Height_(m),PercentFull_Active_Lake_Storage_(%),Snow_Volume_Opuha_Catchment_(mm),WDir(Deg),WSpd(m/s),GustDir(Deg),GustSpd(m/s),WindRun(Km),Rain(mm),Tdry(C),TWet(C),RH(%),Tmax(C),Tmin(C),Pmsl(hPa),Pstn(hPa),B2_AVG,B3_AVG,B4_AVG,B8_AVG,B8A_AVG,B11_AVG,B12_AVG,cs_AVG,cs_cdf_AVG
0,2018-12-14 10:38:15,2068.0,2138.0,2084.0,2288.0,2678.0,2924.0,3312.0,2958.0,2958.0,2362.0,2290.0,2082.0,2336.0,2428.0,2790.0,3060.0,3278.0,2692.0,2248.0,2200.0,2200.0,2240.0,2162.0,2402.0,2330.0,2514.0,2510.0,2700.0,3364.0,3128.0,3128.0,2540.0,2456.0,2004.0,2496.0,2724.0,2946.0,2888.0,3296.0,3046.0,2558.0,2342.0,2342.0,2354.0,2220.0,2348.0,2382.0,2546.0,2256.0,2320.0,3116.0,3096.0,3096.0,2520.0,2394.0,1942.0,2350.0,2656.0,2808.0,2486.0,3128.0,3042.0,2496.0,2244.0,2244.0,2280.0,3210.0,3464.0,3472.0,3662.0,3654.0,3824.0,4516.0,4208.0,4208.0,3694.0,3614.0,3052.0,3654.0,3798.0,4094.0,4078.0,4476.0,4038.0,3552.0,3530.0,3530.0,3540.0,4033.0,3869.0,4052.0,4052.0,3665.0,3665.0,3755.0,4293.0,4293.0,4293.0,3967.0,4033.0,3967.0,4052.0,4052.0,3755.0,3755.0,4389.0,4389.0,3967.0,3967.0,3967.0,3278.0,3069.0,3312.0,3312.0,3308.0,3308.0,3469.0,3662.0,3662.0,3662.0,3326.0,3278.0,3326.0,3312.0,3312.0,3469.0,3469.0,3527.0,3527.0,3326.0,3326.0,3326.0,3077.0,2747.0,2999.0,2999.0,2849.0,2849.0,2939.0,3217.0,3217.0,3217.0,2977.0,3077.0,2977.0,2999.0,2999.0,2939.0,2939.0,3177.0,3177.0,2977.0,2977.0,2977.0,0.164706,0.211765,0.219608,0.215686,0.168627,0.164706,0.160784,0.172549,0.172549,0.180392,0.188235,0.160784,0.200000,0.211765,0.211765,0.160784,0.164706,0.180392,0.188235,

In [5]:
merged_df=merged_df.drop(columns=['Discharge_(m^3/s)'])

In [6]:
merged_df

,timestamp_sentinel2,point1_B2,point10_B2,point11_B2,point12_B2,point13_B2,point14_B2,point15_B2,point16_B2,point17_B2,point18_B2,point19_B2,point2_B2,point20_B2,point21_B2,point22_B2,point3_B2,point4_B2,point5_B2,point6_B2,point7_B2,point8_B2,point9_B2,point1_B3,point10_B3,point11_B3,point12_B3,point13_B3,point14_B3,point15_B3,point16_B3,point17_B3,point18_B3,point19_B3,point2_B3,point20_B3,point21_B3,point22_B3,point3_B3,point4_B3,point5_B3,point6_B3,point7_B3,point8_B3,point9_B3,point1_B4,point10_B4,point11_B4,point12_B4,point13_B4,point14_B4,point15_B4,point16_B4,point17_B4,point18_B4,point19_B4,point2_B4,point20_B4,point21_B4,point22_B4,point3_B4,point4_B4,point5_B4,point6_B4,point7_B4,point8_B4,point9_B4,point1_B8,point10_B8,point11_B8,point12_B8,point13_B8,point14_B8,point15_B8,point16_B8,point17_B8,point18_B8,point19_B8,point2_B8,point20_B8,point21_B8,point22_B8,point3_B8,point4_B8,point5_B8,point6_B8,point7_B8,point8_B8,point9_B8,point1_B8A,point10_B8A,point11_B8A,point12_B8A,point13_B8A,point14_B8A,point15_B8A,point16_B8A,point17_B8A,point18_B8A,point19_B8A,point2_B8A,point20_B8A,point21_B8A,point22_B8A,point3_B8A,point4_B8A,point5_B8A,point6_B8A,point7_B8A,point8_B8A,point9_B8A,point1_B11,point10_B11,point11_B11,point12_B11,point13_B11,point14_B11,point15_B11,point16_B11,point17_B11,point18_B11,point19_B11,point2_B11,point20_B11,point21_B11,point22_B11,point3_B11,point4_B11,point5_B11,point6_B11,point7_B11,point8_B11,point9_B11,point1_B12,point10_B12,point11_B12,point12_B12,point13_B12,point14_B12,point15_B12,point16_B12,point17_B12,point18_B12,point19_B12,point2_B12,point20_B12,point21_B12,point22_B12,point3_B12,point4_B12,point5_B12,point6_B12,point7_B12,point8_B12,point9_B12,point1_cs,point10_cs,point11_cs,point12_cs,point13_cs,point14_cs,point15_cs,point16_cs,point17_cs,point18_cs,point19_cs,point2_cs,point20_cs,point21_cs,point22_cs,point3_cs,point4_cs,point5_cs,point6_cs,point7_cs,point8_cs,point9_cs,point1_cs_cdf,point10_cs_cdf,point11_cs_cdf,point12_cs_cdf,point13_cs_cdf,point14_cs_cdf,point15_cs_cdf,point16_cs_cdf,point17_cs_cdf,point18_cs_cdf,point19_cs_cdf,point2_cs_cdf,point20_cs_cdf,point21_cs_cdf,point22_cs_cdf,point3_cs_cdf,point4_cs_cdf,point5_cs_cdf,point6_cs_cdf,point7_cs_cdf,point8_cs_cdf,point9_cs_cdf,sun_zenith,sun_azimuth,Water_Temp_Buoy_(degC),Water_Temp_Platform_(degC),Turbidity_Buoy_(NTU),Turbidity_Platform_(NTU),Ground_Measurements_time_diff_(seconds),Lake_Height_(m),PercentFull_Active_Lake_Storage_(%),Snow_Volume_Opuha_Catchment_(mm),WDir(Deg),WSpd(m/s),GustDir(Deg),GustSpd(m/s),WindRun(Km),Rain(mm),Tdry(C),TWet(C),RH(%),Tmax(C),Tmin(C),Pmsl(hPa),Pstn(hPa),B2_AVG,B3_AVG,B4_AVG,B8_AVG,B8A_AVG,B11_AVG,B12_AVG,cs_AVG,cs_cdf_AVG
0,2018-12-14 10:38:15,2068.0,2138.0,2084.0,2288.0,2678.0,2924.0,3312.0,2958.0,2958.0,2362.0,2290.0,2082.0,2336.0,2428.0,2790.0,3060.0,3278.0,2692.0,2248.0,2200.0,2200.0,2240.0,2162.0,2402.0,2330.0,2514.0,2510.0,2700.0,3364.0,3128.0,3128.0,2540.0,2456.0,2004.0,2496.0,2724.0,2946.0,2888.0,3296.0,3046.0,2558.0,2342.0,2342.0,2354.0,2220.0,2348.0,2382.0,2546.0,2256.0,2320.0,3116.0,3096.0,3096.0,2520.0,2394.0,1942.0,2350.0,2656.0,2808.0,2486.0,3128.0,3042.0,2496.0,2244.0,2244.0,2280.0,3210.0,3464.0,3472.0,3662.0,3654.0,3824.0,4516.0,4208.0,4208.0,3694.0,3614.0,3052.0,3654.0,3798.0,4094.0,4078.0,4476.0,4038.0,3552.0,3530.0,3530.0,3540.0,4033.0,3869.0,4052.0,4052.0,3665.0,3665.0,3755.0,4293.0,4293.0,4293.0,3967.0,4033.0,3967.0,4052.0,4052.0,3755.0,3755.0,4389.0,4389.0,3967.0,3967.0,3967.0,3278.0,3069.0,3312.0,3312.0,3308.0,3308.0,3469.0,3662.0,3662.0,3662.0,3326.0,3278.0,3326.0,3312.0,3312.0,3469.0,3469.0,3527.0,3527.0,3326.0,3326.0,3326.0,3077.0,2747.0,2999.0,2999.0,2849.0,2849.0,2939.0,3217.0,3217.0,3217.0,2977.0,3077.0,2977.0,2999.0,2999.0,2939.0,2939.0,3177.0,3177.0,2977.0,2977.0,2977.0,0.164706,0.211765,0.219608,0.215686,0.168627,0.164706,0.160784,0.172549,0.172549,0.180392,0.188235,0.160784,0.200000,0.211765,0.211765,0.160784,0.164706,0.180392,0.188235,0.192157,0.192157,

In [7]:
merged_df.to_csv('test_ground_without_discharge.csv', index=False)

# Fix up for the discharge

In [8]:
# Load the data
merged_data_path = 'test_ground_without_discharge.csv'
discharge_data_path = 'Discharge.csv'

# Read the merged dataset
merged_df = pd.read_csv(merged_data_path, parse_dates=['timestamp_sentinel2'])

# Read the discharge data and skip the first row (metadata)
discharge_df = pd.read_csv(discharge_data_path, skiprows=1, parse_dates=['Timestamp (UTC+12:00)'])

# Rename columns for consistency
discharge_df.rename(columns={'Timestamp (UTC+12:00)': 'timestamp_discharge', 'Value (m^3/s)': 'Discharge_(m^3/s)'}, inplace=True)

# Function to find the closest discharge measurement to a given Sentinel-2 timestamp
def find_closest_discharge(sentinel_time, discharge_df):
    time_diffs = (discharge_df['timestamp_discharge'] - sentinel_time).abs()
    closest_idx = time_diffs.idxmin()
    closest_value = discharge_df.at[closest_idx, 'Discharge_(m^3/s)']
    closest_time = discharge_df.at[closest_idx, 'timestamp_discharge']
    time_diff = abs((closest_time - sentinel_time).total_seconds())  # Time difference in seconds
    return closest_value, time_diff

# Prepare a list to store merged data
merged_with_discharge_data = []

# Process each Sentinel-2 timestamp
for index, row in merged_df.iterrows():
    sentinel_time = row['timestamp_sentinel2']
    merged_row = row.to_dict()
    
    # Find the closest discharge measurement
    closest_discharge, discharge_time_diff = find_closest_discharge(sentinel_time, discharge_df)
    merged_row['Discharge_(m^3/s)'] = closest_discharge
    merged_row['Discharge_time_diff_(seconds)'] = discharge_time_diff
    
    merged_with_discharge_data.append(merged_row)

# Convert the merged data to a DataFrame
merged_with_discharge_df = pd.DataFrame(merged_with_discharge_data)

# Save the merged dataframe to a CSV file
merged_with_discharge_df.to_csv('Data_Merged.csv', index=False)

print("Merging completed. The merged dataset with discharge is saved as 'Data_Merged.csv'")

Merging completed. The merged dataset with discharge is saved as 'Merged_Data_with_Discharge.csv'


In [9]:
merged_with_discharge_df

,timestamp_sentinel2,point1_B2,point10_B2,point11_B2,point12_B2,point13_B2,point14_B2,point15_B2,point16_B2,point17_B2,point18_B2,point19_B2,point2_B2,point20_B2,point21_B2,point22_B2,point3_B2,point4_B2,point5_B2,point6_B2,point7_B2,point8_B2,point9_B2,point1_B3,point10_B3,point11_B3,point12_B3,point13_B3,point14_B3,point15_B3,point16_B3,point17_B3,point18_B3,point19_B3,point2_B3,point20_B3,point21_B3,point22_B3,point3_B3,point4_B3,point5_B3,point6_B3,point7_B3,point8_B3,point9_B3,point1_B4,point10_B4,point11_B4,point12_B4,point13_B4,point14_B4,point15_B4,point16_B4,point17_B4,point18_B4,point19_B4,point2_B4,point20_B4,point21_B4,point22_B4,point3_B4,point4_B4,point5_B4,point6_B4,point7_B4,point8_B4,point9_B4,point1_B8,point10_B8,point11_B8,point12_B8,point13_B8,point14_B8,point15_B8,point16_B8,point17_B8,point18_B8,point19_B8,point2_B8,point20_B8,point21_B8,point22_B8,point3_B8,point4_B8,point5_B8,point6_B8,point7_B8,point8_B8,point9_B8,point1_B8A,point10_B8A,point11_B8A,point12_B8A,point13_B8A,point14_B8A,point15_B8A,point16_B8A,point17_B8A,point18_B8A,point19_B8A,point2_B8A,point20_B8A,point21_B8A,point22_B8A,point3_B8A,point4_B8A,point5_B8A,point6_B8A,point7_B8A,point8_B8A,point9_B8A,point1_B11,point10_B11,point11_B11,point12_B11,point13_B11,point14_B11,point15_B11,point16_B11,point17_B11,point18_B11,point19_B11,point2_B11,point20_B11,point21_B11,point22_B11,point3_B11,point4_B11,point5_B11,point6_B11,point7_B11,point8_B11,point9_B11,point1_B12,point10_B12,point11_B12,point12_B12,point13_B12,point14_B12,point15_B12,point16_B12,point17_B12,point18_B12,point19_B12,point2_B12,point20_B12,point21_B12,point22_B12,point3_B12,point4_B12,point5_B12,point6_B12,point7_B12,point8_B12,point9_B12,point1_cs,point10_cs,point11_cs,point12_cs,point13_cs,point14_cs,point15_cs,point16_cs,point17_cs,point18_cs,point19_cs,point2_cs,point20_cs,point21_cs,point22_cs,point3_cs,point4_cs,point5_cs,point6_cs,point7_cs,point8_cs,point9_cs,point1_cs_cdf,point10_cs_cdf,point11_cs_cdf,point12_cs_cdf,point13_cs_cdf,point14_cs_cdf,point15_cs_cdf,point16_cs_cdf,point17_cs_cdf,point18_cs_cdf,point19_cs_cdf,point2_cs_cdf,point20_cs_cdf,point21_cs_cdf,point22_cs_cdf,point3_cs_cdf,point4_cs_cdf,point5_cs_cdf,point6_cs_cdf,point7_cs_cdf,point8_cs_cdf,point9_cs_cdf,sun_zenith,sun_azimuth,Water_Temp_Buoy_(degC),Water_Temp_Platform_(degC),Turbidity_Buoy_(NTU),Turbidity_Platform_(NTU),Ground_Measurements_time_diff_(seconds),Lake_Height_(m),PercentFull_Active_Lake_Storage_(%),Snow_Volume_Opuha_Catchment_(mm),WDir(Deg),WSpd(m/s),GustDir(Deg),GustSpd(m/s),WindRun(Km),Rain(mm),Tdry(C),TWet(C),RH(%),Tmax(C),Tmin(C),Pmsl(hPa),Pstn(hPa),B2_AVG,B3_AVG,B4_AVG,B8_AVG,B8A_AVG,B11_AVG,B12_AVG,cs_AVG,cs_cdf_AVG,Discharge_(m^3/s),Discharge_time_diff_(seconds)
0,2018-12-14 10:38:15,2068.0,2138.0,2084.0,2288.0,2678.0,2924.0,3312.0,2958.0,2958.0,2362.0,2290.0,2082.0,2336.0,2428.0,2790.0,3060.0,3278.0,2692.0,2248.0,2200.0,2200.0,2240.0,2162.0,2402.0,2330.0,2514.0,2510.0,2700.0,3364.0,3128.0,3128.0,2540.0,2456.0,2004.0,2496.0,2724.0,2946.0,2888.0,3296.0,3046.0,2558.0,2342.0,2342.0,2354.0,2220.0,2348.0,2382.0,2546.0,2256.0,2320.0,3116.0,3096.0,3096.0,2520.0,2394.0,1942.0,2350.0,2656.0,2808.0,2486.0,3128.0,3042.0,2496.0,2244.0,2244.0,2280.0,3210.0,3464.0,3472.0,3662.0,3654.0,3824.0,4516.0,4208.0,4208.0,3694.0,3614.0,3052.0,3654.0,3798.0,4094.0,4078.0,4476.0,4038.0,3552.0,3530.0,3530.0,3540.0,4033.0,3869.0,4052.0,4052.0,3665.0,3665.0,3755.0,4293.0,4293.0,4293.0,3967.0,4033.0,3967.0,4052.0,4052.0,3755.0,3755.0,4389.0,4389.0,3967.0,3967.0,3967.0,3278.0,3069.0,3312.0,3312.0,3308.0,3308.0,3469.0,3662.0,3662.0,3662.0,3326.0,3278.0,3326.0,3312.0,3312.0,3469.0,3469.0,3527.0,3527.0,3326.0,3326.0,3326.0,3077.0,2747.0,2999.0,2999.0,2849.0,2849.0,2939.0,3217.0,3217.0,3217.0,2977.0,3077.0,2977.0,2999.0,2999.0,2939.0,2939.0,3177.0,3177.0,2977.0,2977.0,2977.0,0.164706,0.211765,0.219608,0.215686,0.168627,0.164706,0.160784,0.172549,0.172549,0.180392,0.188235,0.160784,0.200000,0.211765,0.211765,0.1607